In [5]:
from os import getenv, listdir
import pandas as pd
from requests import get
from time import sleep

In [6]:
AV_API = getenv('AV_API')

In [7]:
currencies = pd.read_csv('./currency_list.csv')
currencies

,currency_code,currency_name
0,1ST,FirstBlood
1,2GIVE,GiveCoin
2,808,808Coin
3,AAVE,Aave
4,ABT,ArcBlock
...,...,...
570,ZEN,ZenCash
571,ZET,Zetacoin
572,ZIL,Zilliqa
573,ZLA,Zilla


In [8]:
try:
    allData = pd.read_csv('./data/all_data.csv')
except FileNotFoundError:
    allData = pd.DataFrame()
else:
    allData = allData.iloc[: , 1:]

for timeFrame in ('daily', 'weekly', 'monthly'):
    for symbol in currencies['currency_code']:

        if f'{timeFrame}_{symbol}.csv' in listdir('./data/'):
            continue

        url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_{timeFrame.upper()}&symbol={symbol}&market=USD&apikey={AV_API}&datatype=csv'
        response = get(url, timeout = 60)
        sleep(15)

        if 'API' in response.text:
            print(f'{timeFrame} {symbol} reached to API limit.')
            continue

        print(f'{timeFrame} {symbol} {response}')

        if response.status_code == 200:
            with open(f'./data/{timeFrame}_{symbol}.csv', 'wb') as dataFile:
                dataFile.write(response.content)
        else:
            print(f'{timeFrame} {symbol} not responding correctly.')
            continue

        data = pd.read_csv(f'./data/{timeFrame}_{symbol}.csv')
        data['time_frame'] = timeFrame
        data['symbol'] = symbol

        allData = pd.concat([allData, data])
        allData.reset_index(drop = True, inplace = True)
        allData.to_csv('./data/all_data.csv')

allData

daily 1ST reached to API limit.
daily 2GIVE reached to API limit.
daily 808 reached to API limit.
daily AAVE <Response [200]>
daily ABT reached to API limit.
daily ABY reached to API limit.
daily AC reached to API limit.
daily ACT reached to API limit.
daily ADA <Response [200]>
daily ADT reached to API limit.
daily ADX <Response [200]>
daily AE reached to API limit.
daily AEON reached to API limit.
daily AGI reached to API limit.
daily AGRS reached to API limit.
daily AI reached to API limit.
daily AID reached to API limit.
daily AION <Response [200]>
daily AIR reached to API limit.
daily AKY reached to API limit.
daily ALGO <Response [200]>
daily ALIS reached to API limit.
daily AMBER reached to API limit.
daily AMP <Response [200]>
daily AMPL reached to API limit.
daily ANC reached to API limit.
daily ANT <Response [200]>
daily APPC reached to API limit.
daily APX reached to API limit.
daily ARDR reached to API limit.
daily ARK reached to API limit.
daily ARN reached to API limit.
d

ConnectionError: HTTPSConnectionPool(host='www.alphavantage.co', port=443): Max retries exceeded with url: /query?function=DIGITAL_CURRENCY_DAILY&symbol=EOS&market=USD&apikey=8PFYGEGK5TUXR7ZY&datatype=csv (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002394F72EC50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))